# Training Data Evaluation


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from pygsp import graphs, filters, plotting, reduction
import pandas
from enum import Enum
import scipy.spatial
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import seaborn as sns
from sklearn.tree import export_graphviz
import pydot
import pickle
from joblib import dump, load
import os.path

%load_ext autoreload
%autoreload 2
%matplotlib inline

plotting.BACKEND = 'matplotlib'
plt.rcParams['figure.figsize'] = (10, 5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
class SubmapState(Enum):
    ALL_GOOD = 1
    GLOBAL_BAD = 2
    MID_BAD = 3
    LOCAL_BAD = 4
    NO_GOOD = 5

class WaveletEvaluator(object):
    def __init__(self, n_scales = 7):
        self.n_scales = n_scales

    def set_scales(self, n_scales):
        self.n_scales = n_scales

    def compare_signals(self, G, x_1, x_2):

        # Compute the wavelets for each node and scale.
        psi = self.compute_wavelets(G)
        print(f"[WaveletEvaluator] psi = {psi.shape}")

        # Compute the wavelet coefficients for x_1 and x_2.
        W_1 = self.compute_wavelet_coefficients(psi, x_1)
        W_2 = self.compute_wavelet_coefficients(psi, x_2)
        print(f"[WaveletEvaluator] W_1 = {W_1.shape}")
        print(f"[WaveletEvaluator] W_2 = {W_2.shape}")


    def compute_wavelets(self, G):
        print(f"[WaveletEvaluator] Computing wavelets for {self.n_scales} scales.")
        g = filters.Meyer(G, self.n_scales)

        # Evalute filter bank on the frequencies (eigenvalues).
        f = g.evaluate(G.e)
        f = np.expand_dims(f.T, 1)
        psi = np.zeros((G.N, G.N, self.n_scales))

        for i in range(0, G.N):

            # Create a Dirac centered at node i.
            x = np.zeros((G.N,1))
            x[i] = 1

            # Transform the signal to spectral domain.
            s = G._check_signal(x)
            s = G.gft(s)

            # Multiply the transformed signal with filter.
            if s.ndim == 1:
                s = np.expand_dims(s, -1)
            s = np.expand_dims(s, 1)
            s = np.matmul(s, f)

            # Transform back the features to the vertex domain.
            psi[i, :, :] = G.igft(s).squeeze()

        return psi

    def compute_wavelets_coeffs(self, wavelet, x_signal):
        n_values = x_signal.shape[0]
        W = np.zeros((n_values, self.n_scales))
        for i in range(0, n_values):
            for j in range(0, self.n_scales):
                W[i,j] = np.matmul(wavelet[i,:,j].transpose(), x_signal)

        return W

    def check_submap(self, coeffs_1, coeffs_2, submap_ids):
        submap_coeffs_1 = coeffs_1[submap_ids, :]
        submap_coeffs_2 = coeffs_2[submap_ids, :]

        D = self.compute_generic_distance(submap_coeffs_1, submap_coeffs_2)
        return self.evaluate_scales(D)
    
    def compute_features_for_submap(self, coeffs_1, coeffs_2, submap_ids):
        submap_coeffs_1 = coeffs_1[submap_ids, :]
        submap_coeffs_2 = coeffs_2[submap_ids, :]
        
        D = self.compute_distances(submap_coeffs_1, submap_coeffs_2)
                              
        data = np.array([            
            # Euclidean distance.
            np.sum(D[0, 0:2]),
            np.sum(D[0, 2:4]),
            np.sum(D[0, 5:]),            
            
            # Correlation.
            np.sum(D[1, 0:2]),
            np.sum(D[1, 2:4]),
            np.sum(D[1, 5:]),                        
            
            # Cityblock distance.
            np.sum(D[2, 0:2]),
            np.sum(D[2, 2:4]),
            np.sum(D[2, 5:]),
            
            # Chebyshev distance.
            np.sum(D[3, 0:2]),
            np.sum(D[3, 2:4]),
            np.sum(D[3, 5:]),            
        ])
        
        return data
        
    def compute_distances(self, coeffs_1, coeffs_2):
        distances = np.zeros((9, self.n_scales))            
        for j in range(0, self.n_scales):                              
            distances[0, j] = scipy.spatial.distance.euclidean(coeffs_1[:,j], coeffs_2[:,j])            
            distances[1, j] = scipy.spatial.distance.correlation(coeffs_1[:,j], coeffs_2[:,j])
            distances[2, j] = scipy.spatial.distance.cityblock(coeffs_1[:,j], coeffs_2[:,j])         
            distances[3, j] = scipy.spatial.distance.chebyshev(coeffs_1[:,j], coeffs_2[:,j])
            
        return distances

## Load Input from File

In [6]:
dataroot = '/mnt/data/datasets/fgsp/'
datasets = ['gonzen_mission_03_cerberus/', 'lee_h_mission_01_cerberus/']

n_datasets = len(datasets)
opt_signals = []
est_signals = []
opt_trajectory = []
est_trajectory = []
opt_graph_coords = []
opt_graph_adj = []
for ds in datasets:
    ds_path = dataroot + ds
    print(f'Loading data from {ds_path}')
    opt_signal_path = ds_path + 'opt_signal.npy'
    opt_traj_path = ds_path + 'opt_trajectory.npy'
    est_signal_path = ds_path + 'est_signal.npy'
    est_traj_path = ds_path + 'est_trajectory.npy'  
    opt_graph_coords_path = ds_path + 'opt_graph_coords.npy'
    opt_graph_adj_path = ds_path + 'opt_graph_adj.npy'
 
    if os.path.isfile(opt_signal_path) and os.path.isfile(est_signal_path):
        opt_signals.append(np.load(opt_signal_path))
        est_signals.append(np.load(est_signal_path))
        
    if os.path.isfile(opt_traj_path) and os.path.isfile(est_traj_path):
        opt_trajectory.append(np.load(opt_traj_path))
        est_trajectory.append(np.load(est_traj_path))
    
    if os.path.isfile(opt_graph_coords_path) and os.path.isfile(opt_graph_adj_path):
        opt_graph_coords.append(np.load(opt_graph_coords_path))
        opt_graph_adj.append(np.load(opt_graph_adj_path))
    
print(f'=======================================')
n_datasets = len(opt_signals)
n_trajectories = len(opt_trajectory)
assert n_datasets == len(est_signals)
assert n_trajectories == len(opt_trajectory)
print(f'Finished loading {n_datasets} datasets and {n_trajectories} trajectories.')

Loading data from /mnt/data/datasets/fgsp/gonzen_mission_03_cerberus/
Loading data from /mnt/data/datasets/fgsp/lee_h_mission_01_cerberus/
Finished loading 2 datasets and 2 trajectories.


----------------------------------------
## Learn a Random Forest Classifier

In [ ]:
all_data = pandas.DataFrame([])
feature_names = ['Euclidean_L',
                 'Euclidean_B',
                 'Euclidean_H',
                 'Correlation_L',
                 'Correlation_B',
                 'Correlation_H',                 
                 'Manhattan_L',
                 'Manhattan_B',
                 'Manhattan_H',
                 'Chebyshev_L',
                 'Chebyshev_B',
                 'Chebyshev_H']

## Position-based Difference

In [ ]:
def compute_labels_for_signals(est_signal, opt_signal, submap_indices):    
    n_est_signal = len(est_signal)
    n_opt_signal = len(opt_signal)        
    assert n_est_signal == n_opt_signal
    for i in submap_indices:
        diff = np.abs(est_signal[i] - opt_signal[i])
        if diff > 0.9:
            return 1
        
    return 0          

labels = {}
for ds_idx in range(0, n_datasets):
    print(f'Computing data for {datasets[ds_idx]}')
    # Compute wavelets.
    eval = WaveletEvaluator()
    psi = eval.compute_wavelets(graph_per_dataset[ds_idx])
    print(f" psi = {psi.shape}")
    
    W_1 = eval.compute_wavelets_coeffs(psi, est_signals[ds_idx])
    W_2 = eval.compute_wavelets_coeffs(psi, opt_signals[ds_idx])
    
    n_submaps = 40
    n_nodes_in_graph = graph_per_dataset[ds_idx].N
#     submap_size = int(n_nodes_in_graph/n_submaps)
    submap_size = 3
    
    indices = []
    submap_indices = []
    state_signal = np.zeros((n_nodes_in_graph, 1))
    submap_data = []
    submap_labels = []
    
    for i in range(0, n_nodes_in_graph):
        n_indices_in_submap = len(submap_indices)

        # If the submap reached max size, evaluate it.
        if n_indices_in_submap > 0 and n_indices_in_submap % submap_size == 0:        
            features = eval.compute_features_for_submap(W_1, W_2, submap_indices)
            label = compute_labels_for_signals(est_signals[ds_idx], opt_signals[ds_idx], submap_indices)
            
            submap_data.append(features)
            submap_labels.append(label)
            submap_indices = []

        submap_indices.append(i)
    
    submap_data = np.array(submap_data)
    submap_labels = np.array(submap_labels)    
    print(f'labels {submap_labels.shape} and submap_data {submap_data.shape}')
    data=pandas.DataFrame({
        feature_names[0]:submap_data[:,0],
        feature_names[1]:submap_data[:,1],
        feature_names[2]:submap_data[:,2],

        feature_names[3]:submap_data[:,3],
        feature_names[4]:submap_data[:,4],
        feature_names[5]:submap_data[:,5],

        feature_names[6]:submap_data[:,6],
        feature_names[7]:submap_data[:,7],
        feature_names[8]:submap_data[:,8],

        feature_names[9]:submap_data[:,9],
        feature_names[10]:submap_data[:,10],
        feature_names[11]:submap_data[:,11],
        
        'state':submap_labels
    })

    all_data=all_data.append(data)

# Simulated Training Data

## Case 1: All good

In [ ]:
def evaluate_nodes(opt_psi, opt_signal, est_psi, est_signal):
    n_signal = len(opt_signal)
    assert n_signal == len(opt_signal)
#     for i in range(0, n_signal):
        
def create_signal_from_traj(traj):
    traj_origin = traj[0,:]
    pos_signal = (traj - traj_origin).squeeze()
    x_pos = np.linalg.norm(pos_signal, ord=2, axis=1)
    return x_pos

def compute_features(coeffs_1, coeffs_2):
    n_dim = coeffs_1.shape[0]
    assert n_dim == coeffs_2.shape[0]
    
    D = compute_distances(coeffs_1, coeffs_2)
    data = np.array([            
        # Euclidean distance.
        np.sum(D[0, 0:2]),
        np.sum(D[0, 2:4]),
        np.sum(D[0, 5:]),            

        # Correlation.
        np.sum(D[1, 0:2]),
        np.sum(D[1, 2:4]),
        np.sum(D[1, 5:]),

        # Cityblock distance.
        np.sum(D[2, 0:2]),
        np.sum(D[2, 2:4]),
        np.sum(D[2, 5:]),

        # Chebyshev distance.
        np.sum(D[3, 0:2]),
        np.sum(D[3, 2:4]),
        np.sum(D[3, 5:]),            
    ])
    
    return np.nan_to_num(data)

def compute_distances(coeffs_1, coeffs_2):
    n_scales = coeffs_1.shape[1]
    distances = np.zeros((9, n_scales))                
    for j in range(0, n_scales):                              
        distances[0, j] = scipy.spatial.distance.euclidean(coeffs_1[:,j], coeffs_2[:,j])
        distances[1, j] = scipy.spatial.distance.correlation(coeffs_1[:,j], coeffs_2[:,j])
        distances[2, j] = scipy.spatial.distance.cityblock(coeffs_1[:,j], coeffs_2[:,j])         
        distances[3, j] = scipy.spatial.distance.chebyshev(coeffs_1[:,j], coeffs_2[:,j])

    return distances
    
opt_traj = np.array([[0,0,0],[1,0,1],[1,2,1],[2,2,1],[3,3,1]])
opt_graph = graphs.NNGraph(opt_traj, use_flann=False, k=3)
opt_graph.compute_fourier_basis()
opt_graph.plot()

opt_eval = WaveletEvaluator()
opt_psi = opt_eval.compute_wavelets(opt_graph)
W_opt = opt_eval.compute_wavelets_coeffs(opt_psi, opt_signal)

opt_signal = create_signal_from_traj(opt_traj)

sigmas = [0.01, 0.02, 0.03, 0.05]
for sigma in sigmas:
    samples_1 = np.random.normal(0, sigma, G.N)
    samples_2 = np.random.normal(0, sigma, G.N)
    samples_3 = np.random.normal(0, sigma, G.N)
    shifted_est_traj = opt_traj + np.array([samples_1, samples_2, samples_3]).transpose()
    est_signal = create_signal_from_traj(shifted_est_traj)
    
    est_graph = graphs.NNGraph(shifted_est_traj, use_flann=False, k=3)
    est_graph.compute_fourier_basis()
    
    est_eval = WaveletEvaluator()
    est_psi = est_eval.compute_wavelets(est_graph)
        
    W_est = eval.compute_wavelets_coeffs(est_psi, est_signal)    
    
    data = compute_features(W_opt, W_est)
    print(f'wavelet is {W_est.shape}')
    print(f'shape of data: {data.shape}')
    print(f'data: {data}')

-------------------------------

In [ ]:
print(f"Size of all_data = {all_data.shape[0]}")
n_good = all_data[all_data['state'] == 0].shape[0]
n_bad = all_data[all_data['state'] == 1].shape[0]
print(f'We have {n_good} good nodes and {n_bad} bad nodes.')
all_data.head()

In [ ]:
all_data = all_data.fillna(0)
all_data = all_data.replace(float('inf'), 1)

# without JSD
#feature_names = ['Cosine_L', 'Cosine_B', 'Cosine_H','Euclidean_L', 'Euclidean_B', 'Euclidean_H','BrayCurtis_L', 'BrayCurtis_B', 'BrayCurtis_H','Correlation_L', 'Correlation_B', 'Correlation_H', 'Canberra_L', 'Canberra_B', 'Canberra_H', 'Minkowski_L', 'Minkowski_B', 'Minkowski_H', 'Manhattan_L', 'Manhattan_B', 'Manhattan_H', 'Chebyshev_L', 'Chebyshev_B', 'Chebyshev_H']

# Full
feature_names = ['Euclidean_L', 'Euclidean_B', 'Euclidean_H','Correlation_L', 'Correlation_B', 'Correlation_H', 'Manhattan_L', 'Manhattan_B', 'Manhattan_H', 'Chebyshev_L', 'Chebyshev_B', 'Chebyshev_H']


In [ ]:
X=all_data[feature_names]
y=all_data['state']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

dump(clf, '../config/forest.joblib') 
#clf = load('filename.joblib') 

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


In [ ]:
feature_imp = pandas.Series(clf.feature_importances_, index=feature_names).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)

# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()

In [ ]:
# Pull out one tree from the forest
tree = clf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_names, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

## Real Tests

In [ ]:
clf = load('../config/forest.joblib') 

In [ ]:
feature_names = ['Cosine_L', 'Cosine_B', 'Cosine_H','Euclidean_L', 'Euclidean_B', 'Euclidean_H','BrayCurtis_L', 'BrayCurtis_B', 'BrayCurtis_H','Correlation_L', 'Correlation_B', 'Correlation_H', 'Canberra_L', 'Canberra_B', 'Canberra_H', 'JSD_L', 'JSD_B', 'JSD_H', 'Minkowski_L', 'Minkowski_B', 'Minkowski_H', 'Manhattan_L', 'Manhattan_B', 'Manhattan_H', 'Chebyshev_L', 'Chebyshev_B', 'Chebyshev_H']

W_1 = eval.compute_wavelets_coeffs(psi, x_opt)
W_2 = eval.compute_wavelets_coeffs(psi, x_loam)
#W_2 = eval.compute_wavelets_coeffs(psi, x_rovio)

n_submaps = 40
submap_size = int(G.N/n_submaps)

n_all_good = 0
n_low_good = 0
n_high_good = 0
n_no_good = 0

indices = []
submap_indices = []
state_signal = np.zeros((G.N, 1))

submap_data = []
for i in range(0, G.N):
    n_indices_in_submap = len(submap_indices)

    # If the submap reached max size, evaluate it.

    if n_indices_in_submap > 0 and n_indices_in_submap % submap_size == 0:        
        features = eval.compute_features_for_submap(W_1, W_2, submap_indices)
        submap_data.append(features)

        submap_indices = []

    submap_indices.append(i)

submap_data = np.array(submap_data)                
data=pandas.DataFrame({
    feature_names[0]:submap_data[:,0],
    feature_names[1]:submap_data[:,1],
    feature_names[2]:submap_data[:,2],

    feature_names[3]:submap_data[:,3],
    feature_names[4]:submap_data[:,4],
    feature_names[5]:submap_data[:,5],

    feature_names[6]:submap_data[:,6],
    feature_names[7]:submap_data[:,7],
    feature_names[8]:submap_data[:,8],

    feature_names[9]:submap_data[:,9],
    feature_names[10]:submap_data[:,10],
    feature_names[11]:submap_data[:,11],

    feature_names[12]:submap_data[:,12],
    feature_names[13]:submap_data[:,13],
    feature_names[14]:submap_data[:,14],

    feature_names[15]:submap_data[:,15],
    feature_names[16]:submap_data[:,16],
    feature_names[17]:submap_data[:,17],

    feature_names[18]:submap_data[:,18],
    feature_names[19]:submap_data[:,19],
    feature_names[20]:submap_data[:,20],

    feature_names[21]:submap_data[:,21],
    feature_names[22]:submap_data[:,22],
    feature_names[23]:submap_data[:,23],

    feature_names[24]:submap_data[:,24],
    feature_names[25]:submap_data[:,25],
    feature_names[26]:submap_data[:,26]    
})

In [ ]:
data = data.fillna(0)
data = data.replace(float('inf'), 1)

prediction = clf.predict(data)
prediction_prob = clf.predict_proba(data)

print(f"Prediction shape {prediction.shape} and prob shape {prediction_prob.shape}")
len(prediction)